# Rosetta Filter

@Author: 黄健
@E-mail: jian.huang@xtalpi.com

@Proofread: 吴炜坤
@email：weikun.wu@xtalpi.com

参考:

https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Filters/Filters-RosettaScripts

https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Filters/filter_pages/SimpleMetricFilter

</br>

### 1. 为什么需要用Filters？

由于Rosetta采用使用MCMC算法，因此并不是每一个在轨迹中的构象都是“好”的。如果采样是多步，每步都依赖于前一步的结果，当某一步骤中产生的构象是不可靠的，后续的所有采样步骤消耗的计算资源是浪费的，如果有的放矢地设置一些过滤器将初始阶段产生的“劣质”构象丢弃掉，无疑可以加速采样的过程并提高可靠性，这就是Rosetta中设置Filter(过滤模块)的初衷。

举个例子，我们知道在ubiqutin(一种蛋白质)的构象优化过程中保留原构象中保留的K11和E34的盐桥是重要的，因此如果在Repack优化侧链时该盐桥被破坏，那么在后续的能量最小化的过程中就不太可能重新连接起来。如果在这一步中指定了相应的Filter，就可以从多条模拟轨迹中筛选出那些盐桥没有被破坏的构象作为后续采样的Pose输入。

<center><img src="./img/salt_brigde.png" width = "400" height = "300" align=center /> </center>

Rosetta中Filter模块分为两个大类别，SimpleMetricFilter以及其他已经定义好的Filters:
- 在前面的章节中，我们认识了SimpleMetrics，它是Rosetta新一代的数据分析和特征报告工具，可以在计算流程的任意步骤处使用SimpleMetrics去获取当前Pose的特征数据，SimpleMetricFilter以SimpleMetrics输出作为阈值进行筛选。
- Filters的种类多于SimpleMetricFilter，一般会以某些结构上的度量（角度、距离等）作为阈值条件，以决定构象的去留。如在设定盐桥的例子里面，我们可以简单地通过设定两个残基之间的距离应小于4埃，来达到滤掉无特定盐桥构象的目的。

#### 思考

相对于生成大量构象进行筛选，为何不从constraint出发直接添加限制，让能量函数有偏向性到用户所需要的构象？

当然其实我们在之前的章节中介绍过constraint，所以如果我们在一开始就对构象中某些测量性质有要求，直接使用constraint、restraint会比Filter在搜索之后进行过滤的操作更为方便（相当于constraint在上游进行限制，而Filter是在暴力搜索构象后逐步筛选过滤）。这样看来，似乎Filter会比较低效。但往往设定采样的constraints和能量函数的有偏性质不那么好拿捏控制，所以至今Filter仍然是重要的保留模块。

***

### 2.通过xmlobject使用Filters

由于历史原因，基本上每一种Filter都可以通过xmlobject的方式调用(关于RS将在第九章进行详细介绍)，Filter的基本语法如下:

```
<FILTERS>
	<SomeFilterName name="&string" confidence="(1 &Real)" other_option=""/>
</FILTERS>
```

值得注意的是，Filter的定义中的confidence的选项，一般而言所有的Filter的confidence默认值均为“1”，这表示该Filter作为一个任务终止与否的逻辑门，符合条件继续任务，不符合条件终止任务。若设定confidence的值为“0”，此时的Filter的作用类似于SimpleMetrics，仅仅根据规则计算特定的测量值或性质，而不会终止任务。若设定confidence的值介于0~1之间，例如0.5，那么在构象搜索的1-confidence（1-0.5=0.5），即一半的时间里总是为“True”，继续进行任务。

举一个实例进行说明:

In [7]:
# 这里给出一个最基本的Filter定义
# 使用原子间的距离作为筛选条件
from pyrosetta.rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
init()

xml_script = """
<FILTERS>
        <AtomicDistance name="salt_bridge" residue1="11A" atomtype1="Nlys" residue2="34A" atomtype2="OOC" distance="3.0" />
</FILTERS>
"""

# 通过xmlobject生成Filters
xml = XmlObjects.create_from_string(xml_script)
salt_bridge_filter = xml.get_filter("salt_bridge")

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2021.18+release.54b4909cd528ede1d749ea69e8046c244fc797f2 2021-05-04T21:04:43] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r282 2021.18+release.54b4909cd52 54b4909cd528ede1d749ea69e8046c244fc797f2 http://www.pyrosetta.org 2021-05-04T21:04:43
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=1386844316 seed_offset=0 real_seed=1386844316 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=1386844316 RG_type=mt19937
protocols.rosetta_scripts

In [8]:
# 读入初始pose结构
pose = pose_from_pdb("./data/1ubq_clean.pdb")
print(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB
PDB file name: ./data/1ubq_clean.pdb
Total residues: 76
Sequence: MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG
Fold tree:
FOLD_TREE  EDGE 1 76 -1 


In [9]:
# 使用当前filter对pose进行计算，即计算两个原子之间的距离
print(salt_bridge_filter.compute(pose))

3.3466357734297865


In [10]:
# 使用filter判断当前pose是否通过，通过为True，任务继续进行，否则为False，任务终止
salt_bridge_filter.apply(pose)

False

In [12]:
# 我们尝试使用pack和minimization的操作，看看是否能通过该filter？
from pyrosetta.rosetta.protocols import minimization_packing as pack_min
from pyrosetta.rosetta.core.pack.task import TaskFactory, operation

scorefxn = create_score_function( "ref2015" )
tf = TaskFactory()
packer = pack_min.PackRotamersMover()
packer.score_function(scorefxn)

# InitializeFromCommandline会调用初始init（）中的选项
tf.push_back(operation.InitializeFromCommandline())

# RestrictToRepacking可以限制不允许进行design
tf.push_back(operation.RestrictToRepacking())
packer.task_factory(tf)

# 能量最小化
minimizer = pack_min.MinMover()
packer.apply(pose)
minimizer.apply(pose)

core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 1401 rotamers at 76 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015


In [13]:
# 可以看到rosetta优化后的构象符合filter的条件
salt_bridge_filter.apply(pose)

True

In [14]:
# 打印具体的计算数值
salt_bridge_filter.score(pose)

2.3640114111461292

### 3.通过PyRosetta的API使用Filters

读者可以发现，每次使用xmlobject调用filter本质上先要构建出一个RosettaScript对象，会比较慢。用户也可以直接用PyRosetta的API直接调用filter, 也会让代码更加的简洁。

举一个相同的例子，如果使用PyRosetta API来实现代码:

In [65]:
# 使用AtomicDistanceFilter API;
from pyrosetta.rosetta.protocols.simple_filters import AtomicDistanceFilter
from pyrosetta import *
init()

# 读入初始pose结构
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 获取atom type基本信息:
atom_NZ_index = res11.atom_index("NZ")

# 获取1号原子的atomtype所有的信息:
atom_type = res11.atom_type(atom_NZ_index)
print(atom_type)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2021.18+release.54b4909cd528ede1d749ea69e8046c244fc797f2 2021-05-04T21:04:43] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r282 2021.18+release.54b4909cd52 54b4909cd528ede1d749ea69e8046c244fc797f2 http://www.pyrosetta.org 2021-05-04T21:04:43
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=570318775 seed_offset=0 real_seed=570318775 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=570318775 RG_type=mt19937
core.import_pose.import_pose

上述例子可以成功找到某个PDB中原子的名成对应的Rosetta Atom Type, 11号赖氨酸的NZ原子的AtomType为Nlys。

In [66]:
# 原子的AtomType: atom_desig1, atom_desig2
# res1、res2: 残基的PDB名，
# distance_filter = AtomicDistanceFilter(res1=11, res2=34, atom_desig1='NZ', atom_desig2='OE1')
distance_filter = AtomicDistanceFilter(11, 34, 'Nlys', 'OOC', True, True, 3.0)
print(distance_filter.compute(pose))

3.3466357734297865


In [67]:
distance_filter.apply(pose)

False